<a href="https://colab.research.google.com/github/holiday-scott/AppStoreAnalysis/blob/main/App_Store_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Most Profitable Free Apps on Google Play and Apple Store

#### Introduction
This is the first project in the Data Analyst in Python learning pathway with DataQuest.

#### Scenario
I am doing data analysis for a company that builds Android and iOS mobile apps. The company only makes apps that are free to download and install, so the main source of revenue consists of in-app ads. This means revenue for any given app is mostly influenced by the number of users who use the app — the more users that see and engage with the ads, the better. 

#### Goal
To understand what type of apps are likely to attract more users.

#### Dataset
* Apple Store: [This data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) contains more than 7000 Apple iOS mobile application details.
* Play Store: [This data set](https://www.kaggle.com/lava18/google-play-store-apps) contains more than 10000 mobile applicationd details.

## Part 1 - Prepration
First we mount Google Drive and read in the data sets.



In [236]:
# Mount Google.
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
#drive.flush_and_unmount()

# Load reader modules
from csv import reader

# Find out where the data is.
#!ls "gdrive/MyDrive/Colab Notebooks/p1/"

### Google Play Data Set ###
opened_file = open('gdrive/MyDrive/Colab Notebooks/p1/PlayStore.csv')
read_file = reader(opened_file)
android = list(read_file)

android_header = android[0]
android = android[1:]

opened_file = open('gdrive/MyDrive/Colab Notebooks/p1/AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

# Show the first two lines of each data set
print("\n")
print("First two lines of the Apple Store data set")
print("-------------------------------------------")
print(ios_header)
print(ios[1])
print("\n")
print("First two lines of the Google Play Store data set")
print("-------------------------------------------------")
print(android_header)
print(android[1])
print("\n")

Mounted at /content/gdrive


First two lines of the Apple Store data set
-------------------------------------------
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


First two lines of the Google Play Store data set
-------------------------------------------------
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']




## Part 1 Continued
Next, we will need an inspection function to take a look at slices of the data as we clean it up.

In [237]:
# Take data set, start and end point of slice, and count columns as a parameter
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line between rows
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

print(android_header)
print('\n')
explore_data(android, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


## Part 2 - Cleaning the Data

### Deleting Rows
This function shows a row and then asks if you want to deletes that row.

In [238]:
def del_row(data_set, row_index):
  print("This is row ", row_index)
  print(data_set[row_index])
  choice = input("Delete this row? [y/n]")
  if choice == "y": # only one valid answer
    del data_set[row_index]
    print("I have deleted row", row_index)
    #return data_set #use for testing.
  else: # else used so any other character results in no deletion 
    print("No action taken")
    #return data_set #use for testing.


We can now use this function to delete the known error on row **10472 of the Android data set**.

Since I'm not sure if the reported row counts to include the header row or not, we will use this function to inspect the row and delete if it matches the description: "Life Made WI-Fi..."

In [239]:
del_row(android,10472)

This is row  10472
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
Delete this row? [y/n]y
I have deleted row 10472


### Duplicate Data Points
Different versions of different apps, as well as updates, re-releases and scraping errors mean there may well be duplicate data points.

Analysis and visualisation will be helped by ensuring each app only occurs once in the data set.

The Android data set does indeed have multiple duplicates rows for the same application.

### Count Duplicates
This function counts the number of duplicate apps in the data set.

In [240]:
def find_dup(dataset):
  duplicates = []
  uniques = []
  print("Total Rows:",len(dataset))
  for row in dataset:
    #print(len(duplicates)) # debug line
    #print(len(uniques)) # debug line
    name = row[0]
    #print(name) # debug line
    #print(row) # debug line
    if name in uniques:
      duplicates.append(name)
    else:
      uniques.append(name)
  print("Duplicate apps:", len(duplicates))
  print("Unique apps:", len(uniques))
  print("Checksum:", len(duplicates)+len(uniques))
  return

find_dup(android)



Total Rows: 10840
Duplicate apps: 1181
Unique apps: 9659
Checksum: 10840


### Delete Duplicates
Now we know they are there, we need to delete them.

We will use two functions:
* **fill_dict**: Creates a dictionary, where each dictionary key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app.
* **fill_new_dataset**: Use the information stored in the dictionary and create a new data set using the dictionary as an index.

In [241]:

def fill_dict(dataset, name, review):
  u_dict = {}
  for row in dataset:
    app_name = row[name]
    app_review = float(row[review])
         
    # If app not in u_dict then add it.
    if (app_name not in u_dict):
      u_dict[app_name] = app_review
    # If app is in u_dict and higher review, replace it
    elif (app_name in u_dict) and (app_review > u_dict.get(app_name)):
      u_dict[app_name] = app_review
  print("New unique dictionary contains:", len(u_dict), "keys with values.")
  return u_dict

fill_dict(android, 0, 3)
print("\n")


New unique dictionary contains: 9659 keys with values.




In [242]:
# Global scope
clean_android = []
clean_names = []

def fill_new_dataset(index_dict, dataset):
  # Loop through the raw dataset.
  for row in dataset:
    app_name = row[0] # Isolate the name for checking.
    n_reviews = float(row[3])
    # Attempt to match the number of reviews against the index dictionary returned from the fill_dict function.
    # n_reviews is unique in both data sets so we take the high review value we found.
    # Check if name has not been processed already.
    if (index_dict[app_name] == n_reviews) and (app_name not in clean_names):
      clean_android.append(row) # Copy to new clean dataset
      clean_names.append(app_name) # Audit the name to make sure we don't have it already.
  print("New unique dataset contains:", len(clean_android), "rows of data.")
  return clean_android

fill_new_dataset(fill_dict(android, 0, 3), android)
print("\n")
print("Here's a couple of rows of the clean data_set:")
explore_data(clean_android, 5, 7, True)



New unique dictionary contains: 9659 keys with values.
New unique dataset contains: 9659 rows of data.


Here's a couple of rows of the clean data_set:
['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'April 26, 2018', '1.1', '4.0.3 and up']


['Infinite Painter', 'ART_AND_DESIGN', '4.1', '36815', '29M', '1,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'June 14, 2018', '6.1.61.1', '4.2 and up']


Number of rows: 9659
Number of columns: 13


### Non-English Apps
Removing non-English apps is important because the company develops English language apps so we don't want to analyse those from other countries.

We will use the ord(string) to indentity any non-English character by its ascii value (over 127).

3 non-English characters is enough fidelity for our needs, though some may get through.

And so we filter with two nested for loops: one to go through the dataset row by row, and inside that one to cycle through the app name char by char.

In [243]:
# iOS and Android datasets have app names in different columns, so we need to 
# specify which column to look at as a parameter.
def get_english(dataset, name_col):
  non_english_names = [] # Two lists so we can check we have them all.
  english_names = []
  non_english_chars = 0
  print("Total Rows:",len(dataset))
  for row in dataset: # Loop through the data set.
    non_english_chars = 0
    name = row[name_col]
    for char in name: # Loop through each app name in each row.
        if ord(char) > 127:
          non_english_chars += 1 # Increment the counter of non-Eng chars.
    if non_english_chars > 3:
      non_english_names.append(row) # Append to the discard pile if non-Eng.
    else:
      english_names.append(row) # Append to the keep pile if Eng.
  # Quality control. 
  print("Apps named in Non-English:", len(non_english_names))
  print("Apps named in English:", len(english_names))
  print("Checksum:", len(non_english_names)+len(english_names))
  return english_names


eng_android = get_english(clean_android, 0)
print("Android data set now has", len(eng_android), "rows and is called eng_android.")
print("\n")
eng_ios = get_english(ios, 1)
print("iOS data set now has", len(eng_ios), "rows and is called eng_ios.")


Total Rows: 9659
Apps named in Non-English: 45
Apps named in English: 9614
Checksum: 9659
Android data set now has 9614 rows and is called eng_android.


Total Rows: 7197
Apps named in Non-English: 1014
Apps named in English: 6183
Checksum: 7197
iOS data set now has 6183 rows and is called eng_ios.


## Part 3 - Free Apps
The company only builds free apps, so there is no point in analysing paid for apps.

Both datasets now need to be filtered again - this time to keep only free apps.



In [244]:
print(ios_header)
print(android_header)
print("\n")
print(eng_ios[0])
print(eng_android[0])

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


We need to look at:
- Column index **4** in the **eng_ios** dataset
- Column index **6** in the **eng_android** dataset

The function logic is essentially the same as get_english and will result in two new datasets: **free_eng_ios** and **free_eng_android**.

Criteria for the filter are different in each dataset:
- iOS: type is float equal to 0.
- Android: type is string equal to 'Free'.

For simplicity, we'll use two functions:
- get_free_ios()
- get_free_android()

In [245]:
def get_free_ios(dataset):
  paid = []
  free = []
  for app in dataset:
    price = app[4]
    if float(price) > 0.0:
      paid.append(app)
    else:
      free.append(app)
  print("Total apps:", len(dataset))
  print("Paid apps:", len(paid))
  print("Free apps:", len(free))
  print("Checksum:", len(paid) + len(free))
  return free


def get_free_android(dataset):
  paid = []
  free = []
  for app in dataset:
    price = app[6]
    if price == "Paid":
      paid.append(app)
    else:
      free.append(app)
  print("Total apps:", len(dataset))
  print("Paid apps:", len(paid))
  print("Free apps:", len(free))
  print("Checksum:", len(paid) + len(free))
  return free

free_eng_android = get_free_android(eng_android)
print("Free Android apps:", len(free_eng_android))
print("\n")
free_eng_ios = get_free_ios(eng_ios)
print("Free iOS apps:", len(free_eng_ios))
print("\n")



Total apps: 9614
Paid apps: 750
Free apps: 8864
Checksum: 9614
Free Android apps: 8864


Total apps: 6183
Paid apps: 2961
Free apps: 3222
Checksum: 6183
Free iOS apps: 3222




## App Popularity
Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets.

The columns which could be fruitful are:
- iOS: **Genre** (index **11**)
- Android: **Category** (index **1**) and **genre** (index **9**)

### Frequency Table
Analysis will start with a frequency table.

The function **freq_table** will return a dictionary frequency table given dataset and column as a parameter.




In [246]:
def freq_table(dataset, column):
  freq_table = {}
  counter = 0
  for row in dataset:
    if row[column] not in freq_table:
      freq_table[row[column]] = 1
    elif row[column] in freq_table:
      counter = freq_table.get(row[column]) + 1
      freq_table[row[column]] = counter
  return freq_table

android_category_dict = freq_table(free_eng_android, 1)
print("Frequency table created for Android Category")
print("This many rows were parsed:", sum(android_category_dict.values()))
print(android_category_dict)
print("\n")
android_genre_dict = freq_table(free_eng_android, 9)
print("Frequency table created for Android Genre")
print("This many rows were parsed:", sum(android_genre_dict.values()))
print(android_genre_dict)
print("\n")
ios_genre_dict = freq_table(free_eng_ios, 11)
print("Frequency table created for iOS genre")
print("This many rows were parsed:", sum(ios_genre_dict.values()))
print(ios_genre_dict)

Frequency table created for Android Category
This many rows were parsed: 8864
{'ART_AND_DESIGN': 57, 'AUTO_AND_VEHICLES': 82, 'BEAUTY': 53, 'BOOKS_AND_REFERENCE': 190, 'BUSINESS': 407, 'COMICS': 55, 'COMMUNICATION': 287, 'DATING': 165, 'EDUCATION': 103, 'ENTERTAINMENT': 85, 'EVENTS': 63, 'FINANCE': 328, 'FOOD_AND_DRINK': 110, 'HEALTH_AND_FITNESS': 273, 'HOUSE_AND_HOME': 73, 'LIBRARIES_AND_DEMO': 83, 'LIFESTYLE': 346, 'GAME': 862, 'FAMILY': 1676, 'MEDICAL': 313, 'SOCIAL': 236, 'SHOPPING': 199, 'PHOTOGRAPHY': 261, 'SPORTS': 301, 'TRAVEL_AND_LOCAL': 207, 'TOOLS': 750, 'PERSONALIZATION': 294, 'PRODUCTIVITY': 345, 'PARENTING': 58, 'WEATHER': 71, 'VIDEO_PLAYERS': 159, 'NEWS_AND_MAGAZINES': 248, 'MAPS_AND_NAVIGATION': 124}


Frequency table created for Android Genre
This many rows were parsed: 8864
{'Art & Design': 53, 'Art & Design;Creativity': 6, 'Auto & Vehicles': 82, 'Beauty': 53, 'Books & Reference': 190, 'Business': 407, 'Comics': 54, 'Comics;Creativity': 1, 'Communication': 287, 'Datin

### Sorting Frequency Tables
Analysis and visualisation is easier if we can sort the frequency tables.

Sorting by value is most helpful because it helps reveal from the data which apps categories and genres are most popular right now.

The function should do the follow:
- Take a dictionary as input
- Have parameter to allow sorting ascending or descending
- Return a sorted dictionary

The method:
- Write a function to get a corresponding key by value.
- Make a sorted list of the values in the frequency table.
- Loop through the sorted list, put the relevant key, make a new dictionary.

#### Function Get_Key
This helper function makes two parallel lists of keys and values from a dictionary.

This works as a translation table. Get the index for value list, and return the corresponding value for that index from the key list.

Our input dictionaries are:
- android_category_dict - value 57
- android_genre_dict - value 53
- ios_genre_dict - value 106

In [247]:

#Gets a key given a value
def get_key(dict_in, value_in):
  list_vals = list(dict_in.values())
  list_keys = list(dict_in.keys())
  index_position = list(dict_in.values()).index(value_in)
  key = (list_keys[index_position])
  return key

#### Function sort_dict
This is the function that sorts a dictionary in ascending or descendning order using the built-in sorted() function.

The function can return raw numbers or percentages.

In [248]:
def sort_dict(dictionary, desc=True, percent=False):
  sorted_dict = {}
  index_list = sorted(list(dictionary.values()), reverse=desc)
  sum_index_list = sum(index_list)
  for index in index_list:
    value = index
    key = get_key(dictionary, value)    
    if percent:
      value = round((index/sum_index_list)*100, 2)
    else:
      value = index
    index_tuple = (key, value)
    sorted_dict[index_tuple[0]] = index_tuple[1]
  return sorted_dict 

android_category_dict_desc = sort_dict(android_category_dict, desc=True, percent=False)
android_category_dict_desc_percent = sort_dict(android_category_dict, desc=True, percent=True)
print("Android App Categories (Raw / %)")
print(android_category_dict_desc)
print(android_category_dict_desc_percent)
print("\n")
android_genre_dict_desc = sort_dict(android_genre_dict, desc=True, percent=False)
android_genre_dict_desc_percent = sort_dict(android_genre_dict, desc=True, percent=True)
print("Android App Genres (Raw / %)")
print(android_genre_dict_desc)
print(android_genre_dict_desc_percent)
print("\n")
ios_genre_dict_desc = sort_dict(ios_genre_dict, desc=True, percent=False)
ios_genre_dict_desc_percent = sort_dict(ios_genre_dict, desc=True, percent=True)
print("iOS App Genres (Raw / %)")
print(ios_genre_dict_desc)
print(ios_genre_dict_desc_percent)
print("\n")



Android App Categories (Raw / %)
{'FAMILY': 1676, 'GAME': 862, 'TOOLS': 750, 'BUSINESS': 407, 'LIFESTYLE': 346, 'PRODUCTIVITY': 345, 'FINANCE': 328, 'MEDICAL': 313, 'SPORTS': 301, 'PERSONALIZATION': 294, 'COMMUNICATION': 287, 'HEALTH_AND_FITNESS': 273, 'PHOTOGRAPHY': 261, 'NEWS_AND_MAGAZINES': 248, 'SOCIAL': 236, 'TRAVEL_AND_LOCAL': 207, 'SHOPPING': 199, 'BOOKS_AND_REFERENCE': 190, 'DATING': 165, 'VIDEO_PLAYERS': 159, 'MAPS_AND_NAVIGATION': 124, 'FOOD_AND_DRINK': 110, 'EDUCATION': 103, 'ENTERTAINMENT': 85, 'LIBRARIES_AND_DEMO': 83, 'AUTO_AND_VEHICLES': 82, 'HOUSE_AND_HOME': 73, 'WEATHER': 71, 'EVENTS': 63, 'PARENTING': 58, 'ART_AND_DESIGN': 57, 'COMICS': 55, 'BEAUTY': 53}
{'FAMILY': 18.91, 'GAME': 9.72, 'TOOLS': 8.46, 'BUSINESS': 4.59, 'LIFESTYLE': 3.9, 'PRODUCTIVITY': 3.89, 'FINANCE': 3.7, 'MEDICAL': 3.53, 'SPORTS': 3.4, 'PERSONALIZATION': 3.32, 'COMMUNICATION': 3.24, 'HEALTH_AND_FITNESS': 3.08, 'PHOTOGRAPHY': 2.94, 'NEWS_AND_MAGAZINES': 2.8, 'SOCIAL': 2.66, 'TRAVEL_AND_LOCAL': 2.34, 

Now we have six sorted dictionaries for further analysis:
- android_category_dict_desc
- android_category_dict_desc_percent
- android_genre_dict_desc
- android_genre_dict_desc_percent
- ios_genre_dict_desc
- ios_genre_dict_desc_percent

## Analysis
The scope of the analysis is as follows:
- Only English-named apps.
- Only free apps.

### Apple Store Analysis
- **Most common genre**: A very dominant 51% of the free apps on App Store are 'Games.' The next genre is 'Entertainment' but it is not even close with 7.8%
- **Patterns & Trends**: There is a slight bias to art and social networking in the frequency table, but it is not significant given the dominance of games. In other words, there is no indication what kind of game it should be.

### Play Store Analysis
- **Most common category**: 'Family' is the dominant category at 19%. Games comes next, at just under 10%. 
- **Most common genre**: Genres are very distributed. At the most frequent end there is 'Tools' at 8%, 'Entertainment' at 6% and 'Education' at 5%.
- **Patterns & Trends**: There are a lot of genres and all of them are just a percent or so between each other. 

### Recommendations
The company should build:
- A free game with family-friendly themes and PEGI rating.



## Installations as a measure of popularity
Play Store data has the 'Installs' column which we can use to calculate the most-installed genres.

App Store data does not, but we can use the rating_count_column as a proxy because we only need relative popularity, not total popularity.

### Function: avg_genre_install
- Loop through the unique genres 
- Sum the user_rating_count
- Count the number of apps in genre
- Average: total_r_count / total_a_count.

### Columns to analyse
- iOS: prime_genre = -5, total_rating_count = 5
- Android: genres = 9, installs = 5

In [249]:
def avg_genre_install(dataset, genre_col, avg_this_col, desc=True, percent=False):
  genre_table = freq_table(dataset, genre_col)
  avg_genre_table = {}
  for genre in genre_table:
    total_ratings_per_genre = 0
    total_apps_in_genre = 0
    for app in dataset:
      if app[genre_col] == genre:
        total_ratings_per_genre += float(app[avg_this_col])
        total_apps_in_genre += 1
        avg_rating_for_genre = int(total_ratings_per_genre / total_apps_in_genre)
        avg_genre_table[genre] = avg_rating_for_genre
  if percent and desc:
    sorted_avg_genre_table = sort_dict(avg_genre_table, desc=True, percent=True)
  if (not desc) and percent:
    sorted_avg_genre_table = sort_dict(avg_genre_table, desc=False, percent=True)
  if desc and (not percent):
    sorted_avg_genre_table = sort_dict(avg_genre_table, desc=True, percent=False)  
  elif (not percent) and (not desc):
    sorted_avg_genre_table = sort_dict(avg_genre_table, desc=False, percent=False)
  return sorted_avg_genre_table

print("Apple Play Store - average ratings given for each genre as a % of the total giving an indication of which genre is most downloaded.")
ios_avg_genre_ratings_ios = avg_genre_install(free_eng_ios, -5, 5, desc=True, percent=True)
print(ios_avg_genre_ratings_ios)



Apple Play Store - average ratings given for each genre as a % of the total giving an indication of which genre is most downloaded.
{'Navigation': 12.12, 'Reference': 10.55, 'Social Networking': 10.08, 'Music': 8.07, 'Weather': 7.36, 'Book': 5.6, 'Food & Drink': 4.69, 'Finance': 4.43, 'Photo & Video': 4.01, 'Travel': 3.98, 'Shopping': 3.79, 'Health & Fitness': 3.28, 'Sports': 3.24, 'Games': 3.21, 'News': 2.99, 'Productivity': 2.96, 'Utilities': 2.63, 'Lifestyle': 2.32, 'Entertainment': 1.98, 'Business': 1.06, 'Education': 0.99, 'Catalogs': 0.56, 'Medical': 0.09}


### Apple Store Analysis
Previously, the analysis suggested a game with family elements.

However, it's likely there is a selection bias at play here. It is not possible that 50% of apps only make up 3% of the downloads. Probably, people are more moved to rate navigation and reference which have utility, over games which are quickly deleted if not to the downloader's taste.

## Popularity Analysis - Play Store

### Cleaning
That installs column contains strings such as "10,000+" so they need cleaning before we can convert and average them

In [ ]:
test = ["100,000", "10,000plus", "", "1,000+"]

for app in free_eng_android:
  n_installs = app[5]
  n_installs = n_installs.replace(',', '')
  n_installs = n_installs.replace('+', '')
  print(n_installs)
  app[5] = n_installs


Now we can run the **avg_genre_install** function on the android dataset.

In [294]:
print("Google Play Store - average ratings given for each genre as a % of the total giving an indication of which genre is most downloaded.")
avg_genre_ratings_ios = avg_genre_install(free_eng_android, 9, 5, desc=True, percent=False)
print(avg_genre_ratings_ios)


Google Play Store - average ratings given for each genre as a % of the total giving an indication of which genre is most downloaded.
{'Communication': 38456119, 'Adventure;Action & Adventure': 35333333, 'Video Players & Editors': 24947335, 'Social': 23253652, 'Arcade': 22888365, 'Casual': 19569221, 'Puzzle;Action & Adventure': 18366666, 'Photography': 17840110, 'Educational;Action & Adventure': 17016666, 'Productivity': 16787331, 'Racing': 15910645, 'Travel & Local': 14051476, 'Casual;Action & Adventure': 12916666, 'Action': 12603588, 'Strategy': 11199902, 'Tools': 10802461, 'Lifestyle;Pretend Play': 10000000, 'News & Magazines': 9549178, 'Music': 9445583, 'Educational;Pretend Play': 9375000, 'Puzzle;Brain Games': 9280666, 'Word': 9094458, 'Racing;Action & Adventure': 8816666, 'Books & Reference': 8767811, 'Puzzle': 8302861, 'Video Players & Editors;Music & Video': 7500000, 'Shopping': 7036877, 'Role Playing;Action & Adventure': 7000000, 'Casual;Pretend Play': 6957142, 'Entertainment;M

Further analysis is likely to reveal that some major apps like WhatsApp and YouTube are dominating 'Communication' and 'Video Players & Editors'.

It would be wise to remove these outliers to get a better sense of the core apps that we can actually compete against.

# Conclusions
In this project, App Store and Play Store mobile apps which were free and given English language names were analysed with the goal of finding the profile of an app which would compete well in both marketplaces.

Therefore, we would recommend either:
1. A family-friendly game
2. An app based around a popular book

Option 2 is the recommendation we would make based on this analysis due to the much smaller marketplace and therefore more well-defined niche.